In [747]:
import tensorflow as tf
import numpy as np
import data

In [748]:
vocab_size = 1500
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print(len(d))
x, y = d[10:20], d[11:21]
max_word_length = max([len(id_to_token[np.argmax(j)]) for j in d])
print(max_word_length)

1909
12063
15


In [749]:
c_to_i, embedding = data.get_char_embedding()
second_word = id_to_token[np.argmax(x[1])]
third_word = id_to_token[np.argmax(x[2])]
chars2 = [c_to_i[c] for c in second_word]
chars3 = [c_to_i[c] for c in third_word]
print(chars2)
print(chars3)
embed2 = embedding[chars2]
embed3 = embedding[chars3]
to_pad2 = max_word_length - len(chars2)
to_pad3 = max_word_length - len(chars3)
pad2 = np.zeros([to_pad2, embedding.shape[0]])
pad3 = np.zeros([to_pad3, embedding.shape[0]])
reshape = [1, max_word_length, embedding.shape[0], 1]
padded2 = np.reshape(np.r_[embed2, pad2], reshape)
padded3 = np.reshape(np.r_[embed3, pad3], reshape)
print(padded2.shape, padded3.shape)
words = [padded2, padded2]

[23, 7, 71]
[8, 10, 44, 7, 50, 47, 23]
(1, 15, 87, 1) (1, 15, 87, 1)


In [750]:
input_height = max_word_length
input_width = embedding.shape[0]
batch_size = 2
print(input_height, input_width)
filter_heights = [2, 3, 4]
feature_maps = [3, 4, 5]

15 87


In [751]:
def inference(inp):
    with tf.variable_scope('conv1') as scope:        
        weight = tf.get_variable('weights', [filter_heights[0], input_width, 1, feature_maps[0]], initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05))
        conv1 = tf.nn.tanh(tf.nn.conv2d(inp, weight, strides=[1, 1, 1, 1], padding='VALID'))
    pool1 = tf.squeeze(tf.nn.max_pool(conv1, ksize=[1, conv1.get_shape()[1], 1, 1], strides=[1, 1, 1, 1], padding='VALID'))
    
    with tf.variable_scope('conv2') as scope:        
        weight = tf.get_variable('weights', [filter_heights[1], input_width, 1, feature_maps[1]], initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05))
        conv2 = tf.nn.tanh(tf.nn.conv2d(inp, weight, strides=[1, 1, 1, 1], padding='VALID'))
    pool2 = tf.squeeze(tf.nn.max_pool(conv2, ksize=[1, conv2.get_shape()[1], 1, 1], strides=[1, 1, 1, 1], padding='VALID'))
    
    with tf.variable_scope('conv3') as scope:        
        weight = tf.get_variable('weights', [filter_heights[2], input_width, 1, feature_maps[2]], initializer=tf.random_uniform_initializer(minval=-0.05, maxval=0.05))
        conv3 = tf.nn.tanh(tf.nn.conv2d(inp, weight, strides=[1, 1, 1, 1], padding='VALID'))
    pool3 = tf.squeeze(tf.nn.max_pool(conv3, ksize=[1, conv3.get_shape()[1], 1, 1], strides=[1, 1, 1, 1], padding='VALID'))
    
    pool_total = tf.concat(0, [pool1, pool2, pool3], name='total_pool')
    return pool_total

In [752]:
tf.reset_default_graph()
input_ = tf.placeholder(shape=[batch_size, max_word_length, embedding.shape[0], 1], dtype=tf.float32, name = 'cnn_in')
words = tf.split(0, batch_size, input_)
cnn_outputs = []
with tf.variable_scope("CNN") as scope:
    for idx, word in enumerate(words):
#         inp = tf.Variable(word, trainable=False, dtype=tf.float32)
        if idx != 0:
            scope.reuse_variables()
        p = inference(word)
        cnn_outputs.append(p)

In [753]:
# RNN

epochs = 5000
hidden_layer = 128
inp_out_size = sum(feature_maps)
learning_rate = 0.1
num_steps = 15
inp_out_size

12

In [754]:
Wxh = tf.Variable(tf.random_uniform([inp_out_size, hidden_layer], minval=-0.05, maxval=0.05))
Whh = tf.Variable(tf.random_uniform([hidden_layer, hidden_layer], minval=-0.05, maxval=0.05))
Why = tf.Variable(tf.random_uniform([hidden_layer, inp_out_size], minval=-0.05, maxval=0.05))
bh = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
by = tf.Variable(tf.zeros([inp_out_size]), dtype=tf.float32)

In [755]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.tanh((tf.matmul(p, Whh))  + tf.matmul(i, Wxh) + bh)
    h = tf.reshape(h, [hidden_layer])
    return h

In [756]:
# a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
# b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
a = tf.identity(cnn_outputs, name='lstm_in')
b = tf.identity(cnn_outputs, name='lstm_out')

initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b * tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
# for g,v in grad_var_pairs:
#     print(g, v.name)
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -1, 1), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

In [757]:
padded = np.concatenate((padded2, padded3), axis = 0)
print(padded.shape)
sess = tf.Session()
feed = {input_: padded, initial: np.zeros(hidden_layer)}
sess.run(tf.initialize_all_variables())
print(sess.run(outputs, feed_dict=feed))
print(sess.run(loss, feed_dict=feed))

(2, 15, 87, 1)
[[ 0.08330277  0.08346034  0.08336122  0.08314192  0.08326701  0.08319741
   0.08330587  0.08340639  0.08341812  0.08336986  0.08331613  0.08345299]
 [ 0.08335734  0.08348259  0.083432    0.08303896  0.08325277  0.08302888
   0.08319126  0.08354958  0.08350077  0.08339152  0.08336149  0.08341287]]
2.82418
